## Manual Training With `Accelerate`

#### Prepare The Training Environment

In [ ]:
# 1. loading the tokenizer & model
from transformers import AutoTokenizer, AutoModelForSequenceClassification


checkpoint = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
# 2. loading the datasets
from datasets import load_dataset


dataset = load_dataset("glue", "mrpc")

In [ ]:
# 3. tokenizing the datasets
def tokenize_function(example):
  """
  Responsible for tokenizing the datasets.
  """
  return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
# 4. prepare the collate function
from transformers import DataCollatorWithPadding


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# 5. remove non-numerical columns, rename label column to labels and set the format to torch tensors
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

In [ ]:
# 6. batching & padding the datasets
from torch.utils.data import DataLoader


train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator)

eval_dataloader = DataLoader(tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator)

In [ ]:
# ensure that the batching & padding is work as expected
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

In [ ]:
# ensure that the model also work as expected
outputs = model(**batch)

outputs.loss, outputs.logits.shape

#### The Optimizer & Learning Rate Scheduler

In [ ]:
# 7. prepare the AdamW optimizing algorithem
from transformers import AdamW


optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
# 8. prepare the learning rate scheduler
from transformers import get_scheduler


num_epochs = 3

num_training_steps = num_epochs * len(train_dataloader)

lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

print(num_training_steps)

#### The Training Loop

In [ ]:
# 9. prepare the accelerator to enable distributed training on multiple GPUs or TPUs
from accelerate import Accelerator


accelerator = Accelerator()

train_dl, eval_dl, model, optimizer = accelerator.prepare(train_dataloader, eval_dataloader, model, optimizer)

In [ ]:
# 10. start the training loop, and here we are using tqdm library to monitor progress
from tqdm.auto import tqdm


progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dl:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)